# Clustering de perfiles de OkCupid
Este notebook muestra el proceso de análisis, preprocesamiento y entrenamiento de modelos para agrupar perfiles similares de la web de citas OkCupid.

## 1. Setup

Librerías importadas en el proyecto.

In [2]:
import numpy as np
import pandas as pd

import os

from IPython.display import IFrame

try:
    from geopy.geocoders import Nominatim
except ModuleNotFoundError:
    %pip install geopy
    from geopy.geocoders import Nominatim

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

Variables globales declaradas en el proyecto.

In [3]:
RANDOM_SEED = 1337
np.random.seed(RANDOM_SEED)

#!git clone https://github.com/dietrujillo/ml_lab.git ./ml_lab
# os.path.join(os.curdir, "ml_lab", "lab3", "data")  # Path to csv data
DATA_PATH = os.path.join(os.pardir, "data")
# os.path.join(os.curdir, "ml_lab", "lab3", "info")  # Path to problem info
INFO_PATH = os.path.join(os.pardir, "info")
# os.path.join(os.curdir, "ml_lab", "lab3", "results")  # Path to results files
RESULTS_PATH = os.path.join(os.pardir, "results")

Enunciado del problema.

In [3]:
# Jupyter Notebook only
IFrame(os.path.join(INFO_PATH, "lab3.pdf"), width=1080, height=920)

## 2. Análisis exploratorio de los datos

In [6]:
pd.set_option("display.max_columns", 50)
df = pd.read_csv(os.path.join(DATA_PATH, "okcupid_profiles.csv"))
display(df.iloc[:, :10])
display(df.iloc[:, 10:20])
display(df.iloc[:, 20:31])

In [12]:
df.dtypes

age              int64
status          object
sex             object
orientation     object
body_type       object
diet            object
drinks          object
drugs           object
education       object
ethnicity       object
height         float64
income           int64
job             object
last_online     object
location        object
offspring       object
pets            object
religion        object
sign            object
smokes          object
speaks          object
essay0          object
essay1          object
essay2          object
essay3          object
essay4          object
essay5          object
essay6          object
essay7          object
essay8          object
essay9          object
dtype: object

La codificación de los tipos de las variables es correcta, por lo que no será necesario hacer ninguna transformación de tipos. Se tienen variables numéricas, categóricas y textuales, que se analizarán paso por paso. 

In [ ]:
numerical_vars = ["age", "height", "income"]
categorical_vars = ["status", "sex", "orientation", "body_type", "diet", "drinks", "drugs", "education",
                    "ethnicity", "job", "location", "offspring", "pets", "religion", "sign", "smokes", "speaks"]
text_vars = [f"essay{i}" for i in range(10)]

Sin embargo, llegados a este punto podemos prescindir de una variable que no será de utilidad a la hora de hacer el clustering. Esta variable es `last_online`, que repesenta la fecha y hora de última conexión. Con esta decisión reducimos el volumen de datos y nos centramos en las columnas numéricas y categóricas que aportan información sobre la personalidad, preferencias o apariencia física de una persona.

In [71]:
del dataset["last_online"]

### 2.1 Análisis de variables numéricas

#TODO

In [73]:
dataset.describe()

,age,height,income
count,59946.000000,59943.000000,59946.000000
mean,32.340290,68.295281,20033.222534
std,9.452779,3.994803,97346.192104
min,18.000000,1.000000,-1.000000
25%,26.000000,66.000000,-1.000000
50%,30.000000,68.000000,-1.000000
75%,37.000000,71.000000,-1.000000
max,110.000000,95.000000,1000000.000000


En este punto ya podemos observar como hay algunos datos numéricos un tanto extraños:
* La edad máxima es 110, lo cual hace dudar que sea un dato correcto. Posiblemente esta edad corresponda con personas que no quieren compartir su edad verdadera.
* La altura parece estar en pulgadas, *inches*, la media es 68 que se corresponde con 172 cm aprox. En este sentido vemos que la altura mínima es 1 pulgada y que la máxima es de 95 (241 cm). Según la lista de personas más altas del mundo publicada en Wikipedia [1], una altura de entorno a 90 pulgadas(230 cm) ya es algo inaudito.
* Los ingresos llaman la atención en tanto que el ingreso mínimo es -1 y el percentil 75 también es -1. Esto indica que al menos el 75% de los usuarios de la aplicación no han querido decir sus ingresos.

#### 2.2.1 Análisis de la altura

In [74]:
dataset[dataset["height"] > 90]["height"]

402      91.0
2569     95.0
3016     95.0
8089     95.0
12181    93.0
17949    95.0
22358    95.0
23760    94.0
25324    95.0
27506    91.0
33640    95.0
33885    95.0
33946    95.0
34607    95.0
34672    94.0
41939    95.0
43452    95.0
44096    95.0
44264    91.0
47621    92.0
48304    95.0
49194    95.0
51714    95.0
53469    95.0
55172    95.0
56931    95.0
57978    95.0
58147    94.0
Name: height, dtype: float64

#### 2.2.2 Análisis de la edad

In [75]:
dataset[dataset["age"] > 80]

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
2512,110,single,f,straight,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25324,109,available,m,straight,athletic,mostly other,NaN,never,working on masters program,NaN,...,NaN,NaN,NaN,nothing,NaN,NaN,NaN,NaN,NaN,NaN


#### 2.2.3 Análisis de los ingresos

In [76]:
(len(dataset[dataset["income"] < 0]) / len(dataset)) * 100

80.80939512227671

### 2.2 Análisis de las variables categóricas

### 2.3 Análisis de las variables de texto

## 3. Preprocesamiento

## 4. Clustering

## 5. Conclusiones

## 6. Referencias

[1] https://en.wikipedia.org/wiki/List_of_tallest_people

---
Creado por **Diego Trujillo** (diego.trujillo.jimenez@alumnos.upm.es) y **Ángel Acevedo** (angel.acevedo.sanchez@alumnos.upm.es) 

<br></br>
[![License: GPL v2](https://img.shields.io/badge/License-GPL%20v2-blue.svg)](https://www.gnu.org/licenses/old-licenses/gpl-2.0.en.html)